# Task 1 : Warm-up exercise



## Hidden cell: programming

In the cell below we will *import* the code that we're going to want to run.
The syntax for this is
```python
from LOCATION import SOME_CODE
```

The more technical name for `LOCATION` is a *module*, but you can think of this as just a file containing the code we'd like to import.

In the cell below, we are going to import two *classes*.

In [1]:
from p1b_pandemic.lattice import SquareLattice
from p1b_pandemic.model import PandemicModel

You can think of the classes, i.e. `SquareLattice` and `PandemicModel` as flexible templates from which Python is able to create *objects*. In the cell below, we will create an object from each of the two classes.

A real world example will serve us well to explain how classes and objects work. ==TODO==

In [ ]:
lattice = SquareLattice()
model = PandemicModel(lattice)

In the next cell we will tell the model to run an animation.

Note the pair of brackets; these are *crucial*.
By including the brackets, what we are really saying is
> "hello `model`, please check that you contain an function called `animate`, and then *execute* that function."

Without the brackets, all we are saying is
> "hello `model`, please check that you contain an attribute called `animate`."


In [ ]:
model.animate()

## Well that was rubbish...

Fortunately, this is not all we can do.
Remember that classes are *flexible* templates; the behaviour of the objects they create can depend on the *parameters* we provide (i.e. where the 'dials' are set when the object is created).
So far, we have not provided any parameters, so the rubbishy default ones have been used.

The parameters go inside the brackets when we create the objects.
In fact, we have already provided a parameter to `PandemicModel` --- `lattice`.

Let's start by changing the size of the lattice so that it's longer in the horizontal direction.

In [ ]:
dimensions = (25, 100)

lattice = SquareLattice(dimensions)
model = PandemicModel(lattice)

model.animate()

## This still sucks...

Indeed it does.For one thing, the animation did not run for long enough. We will increase the number of updates by providing an argument for `model.animate`.

TODO vaccine frac

In [ ]:
dimensions = (25, 100)
vaccine_frac = 0.1

lattice = SquareLattice(dimensions)
model = PandemicModel(lattice, vaccine_frac=vaccine_frac)

model.animate(100)

## Better but still boring...

What happened here? Clearly, the grey nodes blocked the transmission of 'red-ness'.

If we use quite a deal of imagination, we can try to interpret this in terms of a pandemic.
The story will be that the grey nodes correspond to *immune* units of population, the red ones are *infected* with the virus, and the yellow nodes are the remainder who are *susceptible* to the virus.

You might fairly ask
> in what universe would this be an accurate model of a pandemic??

We can actually be fairly precise about this; one of the advantages of model-building is that it is obvious what your assumptions are.
In this case we assume
* Transmission of the virus occurs with 100% probability between two 'units of population' if one of them is infected and the other is susceptible, and never occurs when one of the units of population is immune.
* Every 'unit of population' has exactly two contacts; one which is able to infect them with the virus, and a second which they are able to transmit to (assuming no immunity).


> What is the probability that the virus is transmitted to the right edge?

There are $n$ nodes in total, of which $k = n * f_\text{vacc}$ are immune.
This is something you can work out analytically.

### Hint




Can we check that the simulation agrees?

Need to do some error analysis...

Get them to understand that we have a binomial distribution for the number of successes.

Mean = prob percolation x num repeats $ = p n $

Variance = sum of individual variances of Bernoulli dist

Standard error is $\sqrt{p(1-p)}.


In [8]:
dimensions = (25, 1000)

lattice = SquareLattice(dimensions)
model = PandemicModel(lattice)

In [11]:
model.vaccine_frac = 0.005
fraction_that_percolated = model.estimate_percolation_prob(repeats=25)
print(fraction_that_percolated)

0.04


In [12]:
model.vaccine_frac = 0.004  # must update *before* re-initialising
model.init_state()
fraction_that_percolated = model.estimate_percolation_prob(repeats=25)
print(fraction_that_percolated)

0.28
